In [1]:
import tweepy
import keys
from textblob import TextBlob
from operator import itemgetter
from wordcloud import WordCloud
import preprocessor as p
import sys
from tweepy import Stream
from tweepy import OAuthHandler
from tweetlistener import TweetListener
from sentimentlistener import SentimentListener
from locationlistener import LocationListener
from tweetutilities import get_geocodes
import pandas as pd
import folium

In [3]:
auth = tweepy.OAuthHandler(keys.TWITTER_CONSUMER_KEY, keys.TWITTER_CONSUMER_SECRET)
auth.set_access_token(keys.ACCESS_TOKEN, keys.ACCESS_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [4]:
#1
mpls_trends = api.get_place_trends(id=2459115)
mpls_list = mpls_trends[0]['trends']
mpls_list = [t for t in mpls_list if t['tweet_volume']]
mpls_list.sort(key=itemgetter('tweet_volume'), reverse=True)
for trend in mpls_list:
    print(trend['name'])

Taylor Swift
NCT 2021
Johnson
Albedo
Jake Gyllenhaal
Red TV
#DisneyPlusDay
Itto
Silk Sonic
Beyoncé
Eula
FOREVER WITH SWEET TAEHYUNG
Good Friday
Mark Meadows
Pence
#FreeBritney
Kenobi
T Swift
MySpace
Bruno Mars
#fridaymorning
Happy Red
#RacistJoe
Predator
bennett
Dani Alves
After Last Night
Damon
Pinocchio
Baymax
Kant


In [23]:
#2
topics = {}
for trend in mpls_list:
     topics[trend['name']] = trend['tweet_volume']
wordcloud = WordCloud(width=1600, height=900,
    prefer_horizontal=0.5, min_font_size=10, colormap='prism',
    background_color='white')
wordcloud = wordcloud.fit_words(topics)
wordcloud = wordcloud.to_file('NewYorkTrending.png')

In [24]:
#3
tweet_listener = SentimentListener(
    consumer_key = keys.TWITTER_CONSUMER_KEY, 
    consumer_secret = keys.TWITTER_CONSUMER_SECRET,
    access_token = keys.ACCESS_TOKEN, 
    access_token_secret = keys.ACCESS_TOKEN_SECRET, limit=100)

In [ ]:
tweet_listener.filter(track=['MySpace'], languages=['en'])

- bellefollevie: That new MySpace gone get a lot of you sneaky ass niggas caught up. Cause y’all are dumb😂and don’t know how to hide shit.

- heather_lenard: Dead 🤣 We all had this hair on MySpace

+ _DCCCVII: What tf is spacehey??? The new MySpace?

  tylah_thegirl: We do NOT need a reboot of myspace....

+ eighty8babyyy: I want my old MySpace back 😩

+ gabyrecabar: I’m real tempted

- YAFYNTHEGOAT: pc4pc bitches “owning” yo pics or getting mad if they wasn’t in yo top friends man that was a era



In [ ]:
tweet_listener.sentiment_dict

In [11]:
#4
tweet_listener = LocationListener(
    consumer_key = keys.TWITTER_CONSUMER_KEY, 
    consumer_secret = keys.TWITTER_CONSUMER_SECRET,
    access_token = keys.ACCESS_TOKEN, 
    access_token_secret = keys.ACCESS_TOKEN_SECRET, limit=100)

In [12]:
tweet_listener.filter(track=['MySpace'], languages=['en'])

IndyCredible: What is this Space Hey business I am seeing ? MySpace pt 2?

IndyCredible: What is this Space Hey business I am seeing ? MySpace pt 2?

IndyCredible: What is this Space Hey business I am seeing ? MySpace pt 2?

JONPFOREVER: RT @SirKedzLaFlare: Myspace could rebrand as a mobile app and take over the game

PimpmikeC: WE BACK!!??? WE BACK?!!

JONPFOREVER: RT @SirKedzLaFlare: Myspace could rebrand as a mobile app and take over the game

JONPFOREVER: RT @SirKedzLaFlare: Myspace could rebrand as a mobile app and take over the game

PimpmikeC: WE BACK!!??? WE BACK?!!

MySideRule: Just to be right back on Twitter 😂

mercyonmarcy: MYSPACE IS BACK!?!?

marialysse: @anikilling @misseverywhereg @_faithskingdom Was born in 99 and never had a myspace

Yung_aku8: RT @SirKedzLaFlare: Myspace could rebrand as a mobile app and take over the game

marialysse: @anikilling @misseverywhereg @_faithskingdom Was born in 99 and never had a myspace

JoelVenomIL: RT @LetsGoHayes_: Was talking to Ji

iJuanaSmoke: RT @stawpfeenin: me setting up my bootleg myspace account on spacehey before it blows up https://t.co/B07l28tt1N

sirparzival24: RT @sumpeoplelikeit: Nobody ever planned a coup or got convinced to eat horse paste on MySpace.

codebymdb: Come be friends with me on the new MySpace: https://t.co/e6ftneSe3W

WallyKhalifah: RT @SirKedzLaFlare: Myspace could rebrand as a mobile app and take over the game

tgadz_: RT @SirKedzLaFlare: Myspace could rebrand as a mobile app and take over the game

almightybankz: RT @SirKedzLaFlare: Myspace could rebrand as a mobile app and take over the game

marathon_luck: RT @RevivalofMan: MySpace was better than any social media today. https://t.co/83P36ATw6W

mujonaderi: RT @_faithskingdom: for my 90s babies☺️ MySpace is back and it’s called SpaceHey 😭💞💕 go add me as a friend at Faithhhhh (faith with 5 h) ht…

yrnrjay: RT @SirKedzLaFlare: Myspace could rebrand as a mobile app and take over the game

scytheslayr: WAIT IS MYSPACE BACK!!!!!



Stream connection closed by Twitter


djtrizzytrip: Niggas making a space hey instead of just going back to MySpace is wild lmao



In [ ]:
tweet_listener.counts_dict

In [14]:
bad_locations = get_geocodes(tweet_listener.tweets_list)

Getting coordinates for tweet locations...
OpenMapQuest service timed out. Waiting.
OpenMapQuest service timed out. Waiting.
OpenMapQuest service timed out. Waiting.
OpenMapQuest service timed out. Waiting.
OpenMapQuest service timed out. Waiting.
Done geocoding


In [21]:
bad_locations

21

In [16]:
df = pd.DataFrame(tweet_listener.tweets_list)
df = df.dropna()

In [17]:
usmap = folium.Map(location=[39.8283, -98.5795], tiles='Stamen Terrain',zoom_start=5, detect_retina=True)

In [18]:
for t in df.itertuples():
    text = ': '.join([t.screen_name, t.text])
    popup = folium.Popup(text, parse_html=True)
    marker = folium.Marker((t.latitude, t.longitude),popup=popup)
    marker.add_to(usmap)

In [19]:
usmap.save('tweet_map1.html')